In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/GitHub/MacroEconBrasil`


In [27]:
using DataFrames
using StatsBase
using Vizagrams
using StructArrays
using KernelDensity
using InverseFunctions
using PrettyNumbers
using CSV
using Dates

In [77]:
ipca = CSV.read("IPCA.csv", DataFrame);
ipca = copy(ipca[:,Not(:Column1)]);
ipcawide = unstack(ipca, :variable, :value)

ipca12 = CSV.read("IPCA_12m.csv", DataFrame);
ipca12 = copy(ipca12[:,Not(:Column1)]);

ipcagp = CSV.read("IPCA_GP.csv", DataFrame);
ipcagp = copy(ipcagp[:,Not(:Column1)]);

In [78]:
yearmonth.(ipca12.date)
ipca12[map(x->yearmonth.(x) == (2023,2), ipca12.date),:]
ipca12[map(x->yearmonth.(x) == (2023,1), ipca12.date),:]

Row,value,date,variable
,Float64,Date,String15
1,5.77,2023-01-01,Var. MM12M (%)


In [109]:
function taxa_correcao_ipca(ano_i,ano_f)
    acc_ano = map(i->ipcawide[map(x->yearmonth.(x) == (i,12), ipcawide.date),"Var. acumulada no ano (%)"][begin], ano_i:ano_f)
    return foldl((acc,s)->acc*(1+s/100),acc_ano,init=100)/100
end

taxa_correcao_ipca (generic function with 1 method)

In [114]:
taxa_correcao_ipca(2020,2023)

1.2731754107310576

In [79]:
v = 100 * (1+5.79/100)
v = v * (1+4.62/100)

110.67749800000001

In [98]:
acc_ano = map(i->ipcawide[map(x->yearmonth.(x) == (i,12), ipcawide.date),"Var. acumulada no ano (%)"][begin], 2010:2023)

14-element Vector{Float64}:
  5.91
  6.5
  5.84
  5.91
  6.41
 10.67
  6.29
  2.95
  3.75
  4.31
  4.52
 10.06
  5.79
  4.62

In [108]:
foldl((acc,s)->acc*(1+s/100),acc_ano,init=100)/100

2.2449505508575074

In [106]:
(foldl((acc,s)->acc*(1+s/100),acc_ano,init=100) - 224.46)*100/224.46

0.015617520159811563

In [80]:
ipca[map(x->yearmonth.(x) == (2020,12), ipca.date),:]

Row,value,date,variable
,Float64,Date,String31
1,1.35,2020-12-01,Var. mensal (%)
2,4.52,2020-12-01,Var. acumulada no ano (%)
3,3.13,2020-12-01,Var. MM3M (%)
4,4.42,2020-12-01,Var. MM6M (%)
5,4.52,2020-12-01,Var. MM12M (%)


In [67]:
ipca[map(x->yearmonth.(x) == (2023,12), ipca.date),:]

Row,value,date,variable
,Float64,Date,String31
1,0.56,2023-12-01,Var. mensal (%)
2,4.62,2023-12-01,Var. acumulada no ano (%)
3,1.08,2023-12-01,Var. MM3M (%)
4,1.7,2023-12-01,Var. MM6M (%)
5,4.62,2023-12-01,Var. MM12M (%)


In [65]:
ipcagp

Row,date,groups,peso,variacao,contribuicao
,Date,String31,Float64,Float64,Float64
1,2020-01-01,1.Alimentação e bebidas,19.3483,0.39,0.0754584
2,2020-01-01,2.Habitação,15.5945,0.55,0.0857698
3,2020-01-01,3.Artigos de residência,3.7529,-0.07,-0.00262703
4,2020-01-01,4.Vestuário,4.577,-0.48,-0.0219696
5,2020-01-01,5.Transportes,20.5979,0.32,0.0659133
6,2020-01-01,6.Saúde e cuidados pessoais,13.5334,-0.32,-0.0433069
7,2020-01-01,7.Despesas pessoais,10.733,0.35,0.0375655
8,2020-01-01,8.Educação,6.1485,0.16,0.0098376
9,2020-01-01,9.Comunicação,5.7146,0.12,0.00685752


In [16]:
ipca12[1:5,:]

Row,Column1,value,date,variable
,Int64,Float64,Date,String15
1,1320,3.02,2007-02-01,Var. MM12M (%)
2,1325,2.96,2007-03-01,Var. MM12M (%)
3,1330,3.0,2007-04-01,Var. MM12M (%)
4,1335,3.18,2007-05-01,Var. MM12M (%)
5,1340,3.69,2007-06-01,Var. MM12M (%)


In [17]:
ipcagp[1:5,:]

Row,Column1,date,groups,peso,variacao,contribuicao
,Int64,Date,String31,Float64,Float64,Float64
1,0,2020-01-01,1.Alimentação e bebidas,19.3483,0.39,0.0754584
2,1,2020-01-01,2.Habitação,15.5945,0.55,0.0857698
3,2,2020-01-01,3.Artigos de residência,3.7529,-0.07,-0.00262703
4,3,2020-01-01,4.Vestuário,4.577,-0.48,-0.0219696
5,4,2020-01-01,5.Transportes,20.5979,0.32,0.0659133
